/data/tensorflow-for-poets-2/tf_files

In [1]:
import os
import sys
import glob
import argparse
from keras import __version__
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping


import numpy as np
import requests
from io import BytesIO

from PIL import Image,ImageDraw,ImageFont
from keras.preprocessing import image
from keras.models import load_model



Using TensorFlow backend.


In [2]:
import tensorflow as tf
from keras import backend
backend.tensorflow_backend._get_available_gpus()


['/job:localhost/replica:0/task:0/device:GPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:1']

In [3]:
IM_WIDTH, IM_HEIGHT = 299, 299 #fixed size for InceptionV3
NB_EPOCHS = 3
BAT_SIZE = 32
FC_SIZE = 1024
NB_IV3_LAYERS_TO_FREEZE = 172

In [4]:
train_data_dir = "/root/finalproject/tensorflow-for-poets-2/tf_files/Training"

In [5]:
# train_datagen = ImageDataGenerator(
# preprocessing_function=preprocess_input,
# rotation_range=30,
# width_shift_range=0.2,
# height_shift_range=0.2,
# shear_range=0.2,
# zoom_range=0.2,
# horizontal_flip=True
# )


In [6]:
# train_generator = train_datagen.flow_from_directory(
# train_data_dir,
# target_size=(IM_WIDTH, IM_HEIGHT),
# batch_size = BAT_SIZE, 
# class_mode = "categorical")


Found 28707 images belonging to 7 classes.


In [5]:
def get_nb_files(directory):
    """Get number of files by searching directory recursively"""
    if not os.path.exists(directory):
        return 0
    cnt = 0
    for r, dirs, files in os.walk(directory):
        for dr in dirs:
            cnt += len(glob.glob(os.path.join(r, dr + "/*")))
    return cnt

In [6]:
def setup_to_transfer_learn(model, base_model):
    """Freeze all layers and compile the model"""
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
def add_new_last_layer(base_model, nb_classes):
    """Add last layer to the convnet
    Args:
    base_model: keras model excluding top
    nb_classes: # of classes
    Returns:
    new keras model with last layer
    """
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(FC_SIZE, activation='relu')(x) #new FC layer, random init
    predictions = Dense(nb_classes, activation='softmax')(x) #new softmax layer
    model = Model(input=base_model.input, output=predictions)
    return model

In [8]:
def setup_to_finetune(model):
    """Freeze the bottom NB_IV3_LAYERS and retrain the remaining top layers.
    note: NB_IV3_LAYERS corresponds to the top 2 inception blocks in the inceptionv3 arch
    Args:
    model: keras model
    """
    for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
        layer.trainable = False
    for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
        layer.trainable = True
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
def train(train_dir, nb_epoch,batch_size, out_dir):
    """Use transfer learning and fine-tuning to train a network on a new dataset"""
    nb_train_samples = get_nb_files(train_dir)
    nb_classes = 7
#     nb_val_samples = get_nb_files(args.val_dir)
    nb_epoch = int(nb_epoch)
    batch_size = int(batch_size)
    # data prep
    train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )
    
#     test_datagen = ImageDataGenerator(
#     preprocessing_function=preprocess_input,
#     rotation_range=30,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True
#     )
    
    train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IM_WIDTH, IM_HEIGHT),
    batch_size=batch_size,
    )
#     validation_generator = test_datagen.flow_from_directory(
#     args.val_dir,
#     target_size=(IM_WIDTH, IM_HEIGHT),
#     batch_size=batch_size,
#     )
    # setup model
    base_model = InceptionV3(weights='imagenet', include_top=False) #include_top=False excludes final FC layer
    model = add_new_last_layer(base_model, nb_classes)
    # transfer learning
    setup_to_transfer_learn(model, base_model)
    
        # Save the model according to the conditions  
    checkpoint = ModelCheckpoint(out_dir, monitor='acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    early = EarlyStopping(monitor='acc', min_delta=0, patience=10, verbose=1, mode='auto')

    
    
    history_tl = model.fit_generator(
    train_generator,
    nb_epoch=nb_epoch,
    samples_per_epoch=nb_train_samples,
#     validation_data=validation_generator,
#     nb_val_samples=nb_val_samples,
    class_weight='auto',
    callbacks = [checkpoint, early])
    
    # fine-tuning
    setup_to_finetune(model)
    history_ft = model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    nb_epoch=nb_epoch,
#     validation_data=validation_generator,
#     nb_val_samples=nb_val_samples,
    class_weight='auto',
    callbacks = [checkpoint, early])
    
    model.save(out_dir)
 
 

In [10]:
IM_WIDTH, IM_HEIGHT = 299, 299 #fixed size for InceptionV3
NB_EPOCHS = 50
BAT_SIZE = 10
FC_SIZE = 1024
NB_IV3_LAYERS_TO_FREEZE = 172

In [ ]:
train(train_dir = train_data_dir, nb_epoch= NB_EPOCHS, batch_size=BAT_SIZE, out_dir = "/data/inception" )

Found 28707 images belonging to 7 classes.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=2870, callbacks=[<keras.ca..., epochs=50, class_weight="auto")`


Epoch 1/50
2870/2870 [==============================] - 614s 214ms/step - loss: 2.3123 - acc: 0.2679

Epoch 00001: acc improved from -inf to 0.26791, saving model to /data/inception
Epoch 2/50
2870/2870 [==============================] - 619s 216ms/step - loss: 1.7207 - acc: 0.3037

Epoch 00002: acc improved from 0.26791 to 0.30373, saving model to /data/inception
Epoch 3/50
2870/2870 [==============================] - 612s 213ms/step - loss: 1.7062 - acc: 0.3137

Epoch 00003: acc improved from 0.30373 to 0.31369, saving model to /data/inception
Epoch 4/50
2870/2870 [==============================] - 610s 213ms/step - loss: 1.6956 - acc: 0.3203

Epoch 00004: acc improved from 0.31369 to 0.32028, saving model to /data/inception
Epoch 5/50
2870/2870 [==============================] - 611s 213ms/step - loss: 1.6912 - acc: 0.3244

Epoch 00005: acc improved from 0.32028 to 0.32446, saving model to /data/inception
Epoch 6/50
2870/2870 [==============================] - 611s 213ms/step - loss

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [12]:
def continue_train(train_dir, nb_epoch,batch_size, out_dir):
    """Use transfer learning and fine-tuning to train a network on a new dataset"""
    nb_train_samples = get_nb_files(train_dir)
    nb_classes = 7
#     nb_val_samples = get_nb_files(args.val_dir)
    nb_epoch = int(nb_epoch)
    batch_size = int(batch_size)
    # data prep
    train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )
    
#     test_datagen = ImageDataGenerator(
#     preprocessing_function=preprocess_input,
#     rotation_range=30,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True
#     )
    
    train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IM_WIDTH, IM_HEIGHT),
    batch_size=batch_size,
    )
#     validation_generator = test_datagen.flow_from_directory(
#     args.val_dir,
#     target_size=(IM_WIDTH, IM_HEIGHT),
#     batch_size=batch_size,
#     )
    # setup model
    base_model = InceptionV3(weights='imagenet', include_top=False) #include_top=False excludes final FC layer
    model = add_new_last_layer(base_model, nb_classes)
    # transfer learning
    setup_to_transfer_learn(model, base_model)
    
        # Save the model according to the conditions  
    checkpoint = ModelCheckpoint(out_dir, monitor='acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    early = EarlyStopping(monitor='acc', min_delta=0, patience=10, verbose=1, mode='auto')

    cont_model = load_model("/data/inception")
    
    history_tl = cont_model.fit_generator(
    train_generator,
    nb_epoch=nb_epoch,
    samples_per_epoch=nb_train_samples,
#     validation_data=validation_generator,
#     nb_val_samples=nb_val_samples,
    class_weight='auto',
    callbacks = [checkpoint, early])
    
    # fine-tuning
    setup_to_finetune(cont_model)
    history_ft = cont_model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    nb_epoch=nb_epoch,
#     validation_data=validation_generator,
#     nb_val_samples=nb_val_samples,
    class_weight='auto',
    callbacks = [checkpoint, early])
    
    model.save(out_dir)
 
 

In [ ]:
continue_train(train_dir = train_data_dir, nb_epoch= 15, batch_size=BAT_SIZE, out_dir = "/data/inception" )

Found 28707 images belonging to 7 classes.


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., class_weight="auto", steps_per_epoch=2870, callbacks=[<keras.ca..., epochs=15)`


Epoch 1/15
2870/2870 [==============================] - 609s 212ms/step - loss: 1.6790 - acc: 0.3383

Epoch 00001: acc improved from -inf to 0.33826, saving model to /data/inception
Epoch 2/15
2870/2870 [==============================] - 604s 211ms/step - loss: 1.6731 - acc: 0.3368

Epoch 00002: acc did not improve from 0.33826
Epoch 3/15
2870/2870 [==============================] - 606s 211ms/step - loss: 1.6762 - acc: 0.3369

Epoch 00003: acc did not improve from 0.33826
Epoch 4/15
2870/2870 [==============================] - 614s 214ms/step - loss: 1.6773 - acc: 0.3380

Epoch 00004: acc did not improve from 0.33826
Epoch 5/15
2870/2870 [==============================] - 616s 215ms/step - loss: 1.6755 - acc: 0.3398

Epoch 00005: acc improved from 0.33826 to 0.33976, saving model to /data/inception
Epoch 6/15
2870/2870 [==============================] - 603s 210ms/step - loss: 1.6828 - acc: 0.3392

Epoch 00006: acc did not improve from 0.33976
Epoch 7/15
2870/2870 [===================

In [11]:
def continue_finetune(train_dir, nb_epoch,batch_size, out_dir):
    """Use transfer learning and fine-tuning to train a network on a new dataset"""
    nb_train_samples = get_nb_files(train_dir)
    nb_classes = 7
#     nb_val_samples = get_nb_files(args.val_dir)
    nb_epoch = int(nb_epoch)
    batch_size = int(batch_size)
    # data prep
    train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )
    
#     test_datagen = ImageDataGenerator(
#     preprocessing_function=preprocess_input,
#     rotation_range=30,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True
#     )
    
    train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IM_WIDTH, IM_HEIGHT),
    batch_size=batch_size,
    )
#     validation_generator = test_datagen.flow_from_directory(
#     args.val_dir,
#     target_size=(IM_WIDTH, IM_HEIGHT),
#     batch_size=batch_size,
#     )
    # setup model
    base_model = InceptionV3(weights='imagenet', include_top=False) #include_top=False excludes final FC layer
    model = add_new_last_layer(base_model, nb_classes)
    # transfer learning
    setup_to_transfer_learn(model, base_model)
    
        # Save the model according to the conditions  
    checkpoint = ModelCheckpoint(out_dir, monitor='acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    early = EarlyStopping(monitor='acc', min_delta=0, patience=10, verbose=1, mode='auto')

    cont_model = load_model("/data/inception")
    
    # fine-tuning
    setup_to_finetune(cont_model)
    history_ft = cont_model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    nb_epoch=nb_epoch,
#     validation_data=validation_generator,
#     nb_val_samples=nb_val_samples,
    class_weight='auto',
    callbacks = [checkpoint, early])
    
 
 

In [12]:
continue_finetune(train_dir = train_data_dir, nb_epoch= 15, batch_size=BAT_SIZE, out_dir = "/data/inception" )

Found 28707 images belonging to 7 classes.
Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  del sys.path[0]


Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:60: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:60: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=2870, class_weight="auto", callbacks=[<keras.ca..., epochs=15)`


Epoch 1/15
2870/2870 [==============================] - 619s 216ms/step - loss: 1.6005 - acc: 0.3783

Epoch 00001: acc improved from -inf to 0.37833, saving model to /data/inception
Epoch 2/15
2870/2870 [==============================] - 608s 212ms/step - loss: 1.4970 - acc: 0.4271

Epoch 00002: acc improved from 0.37833 to 0.42708, saving model to /data/inception
Epoch 3/15
2870/2870 [==============================] - 608s 212ms/step - loss: 1.4402 - acc: 0.4543

Epoch 00003: acc improved from 0.42708 to 0.45433, saving model to /data/inception
Epoch 4/15
2870/2870 [==============================] - 609s 212ms/step - loss: 1.3984 - acc: 0.4664

Epoch 00004: acc improved from 0.45433 to 0.46639, saving model to /data/inception
Epoch 5/15
2870/2870 [==============================] - 609s 212ms/step - loss: 1.3642 - acc: 0.4834

Epoch 00005: acc improved from 0.46639 to 0.48336, saving model to /data/inception
Epoch 6/15
2870/2870 [==============================] - 609s 212ms/step - loss

In [ ]:
checkpoint = ModelCheckpoint(out_dir, monitor='acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='acc', min_delta=0, patience=10, verbose=1, mode='auto')



history_tl = new_model.fit_generator(
train_generator,
nb_epoch=nb_epoch,
samples_per_epoch=nb_train_samples,
#     validation_data=validation_generator,
#     nb_val_samples=nb_val_samples,
class_weight='auto',
callbacks = [checkpoint, early])
    

In [ ]:
 
    
    — train_dir flower_photos/train \
 — val_dir flower_photos/validation \ 
 — nb_epoch 50 \ 
 — batch_size 10 \ 
 — output_model_file inception_yo1.model

In [17]:

import numpy as np
import requests
from io import BytesIO

from PIL import Image,ImageDraw,ImageFont
from keras.preprocessing import image
from keras.models import load_model


In [ ]:

def predict(model, img, target_size):
    """Run model prediction on image
    Args:
    model: keras model
    img: PIL format image
    target_size: (w,h) tuple
    Returns:
    list of predicted labels and their probabilities
    """
    if img.size != target_size:
    img = img.resize(target_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    return preds[0]

if __name__=="__main__":
 a = argparse.ArgumentParser()
 a.add_argument(" - image", help="path to image")
 a.add_argument(" - image_url", help="url to image")
 a.add_argument(" - model")
 args = a.parse_args()
 
 if args.image is None and args.image_url is None:
 a.print_help()
 sys.exit(1)
model = load_model(args.model)
 model.fit()
 if args.image is not None:
 labels = ("daisy", "dandelion","roses","sunflower","tulips")
 image1 = Image.open(args.image)
 preds = predict(model, image1, target_size) 
 print(preds)
 preds = preds.tolist()
 plot_preds(image1, preds)
 fonttype = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",18)
 draw = ImageDraw.Draw(image1)
 draw.text(xy=(5,5),text = str(labels[preds.index(max(preds))])+":"+str(max(preds)),fill = (255,255,255,128),font = fonttype)
 image1.show()
 image1.save((args.image).split(".")[0]+"1"+".jpg")